In [341]:
import pandas as pd
import nltk
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import numpy as np
from ipynb.fs.full.Preprocessing_and_EDA_Capstone_Project import *
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import roc_curve, accuracy_score,f1_score,classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [342]:
import imblearn
print(imblearn.__version__)

0.9.0


In [343]:
industry_data = pd.read_csv("cleaned_industry_safety_data.csv")

In [344]:
industry_data.head()

,Unnamed: 0,001,001 1710,001 1710 ob1,0010,0010 time,0010 time when,007,007 bp105,007 bp105 parking,...,zone intersection rpa,zone lifting,zone lifting container,zone moment,zone moment that,zone the,zone the victim,zone with,zone with the,Accident_Level
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [345]:
industry_data.drop(['Unnamed: 0'],axis =1,inplace = True)
industry_data.head(2)

,001,001 1710,001 1710 ob1,0010,0010 time,0010 time when,007,007 bp105,007 bp105 parking,01022017,...,zone intersection rpa,zone lifting,zone lifting container,zone moment,zone moment that,zone the,zone the victim,zone with,zone with the,Accident_Level
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [346]:
industry_data.shape

(1545, 32284)

In [347]:
industry_data['Accident_Level'].value_counts()

0    309
3    309
2    309
1    309
4    309
Name: Accident_Level, dtype: int64

In [349]:
X = industry_data.drop(['Accident_Level'],axis = 1)

In [350]:
X.shape

(1545, 32283)

In [351]:
X = np.array(X)
y = industry_data['Accident_Level']

In [354]:
print(X.shape,y.shape)

(1545, 32283) (1545,)


In [367]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

### Testing for user input data

In [373]:
def test_user_data(classifier,test_description):
    test_vector = count_vect.transform([test_description])
    #test_scaled = scaler.fit_transform(test_vector.reshape(1,300))
    n_features = industry_data.shape[1]
    test_vector = test_vector[:,0:32283]
    print(test_vector.shape)
    
    test_pred = classifier.predict(test_vector.toarray())
    print(test_pred)
    predictions_test = encoder.inverse_transform(test_pred)
    print(predictions_test)


In [374]:
#models = ['MultinomialNB','GuassianNB','DecisionTreeClassifier','KNeighborsClassifier']
classifiers = [ GaussianNB(),                
                KNeighborsClassifier(),
                DecisionTreeClassifier(),
                RandomForestClassifier(),
                AdaBoostClassifier(),
                LogisticRegression()]

In [375]:
scores_df = pd.DataFrame(index = ['accuracy','precision','recall','f1_score'])
#report_df = pd.DataFrame()
report_df = pd.DataFrame()


In [376]:
heading = ['classifier','class','precision','recall','f1_score','support']
dataframe = pd.DataFrame([],columns = heading)

def classification_report_csv(classifier,report):
    report_data = []
    
    dataframe = pd.DataFrame()
    lines = report.split('\n')
    for line in lines[2:7]:
        row = {}
        row_data = line.split('     ')
        #print(row_data)
        row[str(classifier)] = str(classifier)
        row['class'] = row_data[2]
        #print(row_data[4])
        row['precision'] = (row_data[3])
        row['recall'] = (row_data[4])
        row['f1_score'] = (row_data[5])
        row['support'] = (row_data[6])
        report_data.append(row)
        #print(row)
        #print(report_data)
    dataframe =pd.DataFrame(report_data)
    print(dataframe)
    return dataframe   


In [378]:
report_data = {}
df = pd.DataFrame()
report_df = pd.DataFrame()
def train_model(classifier, X_train, y_train, X_test, y_test,report_df):   
    classifier.fit(X_train,y_train)
    predictions = classifier.predict(X_test)   
    accuracy_score = metrics.accuracy_score(predictions,y_test)
    precision = metrics.precision_score(predictions,y_test,average='micro')
    recall = metrics.recall_score(predictions,y_test,average='micro')
    f1_score = metrics.f1_score(predictions,y_test,average='micro')
    print(str(classifier),'accuracy:',accuracy_score,'precision:',precision,'recall: ',recall,'f1_score: ',f1_score)
    scores_df[str(classifier)] = [accuracy_score,precision,recall,f1_score]
    report = metrics.classification_report(predictions,y_test)
    print(report)
    df = classification_report_csv(classifier,report)    
    #print('Testing for description '+ test_desc)
    #test_user_data(classifier,test_desc)
    #print("True Label ",acc_level)    
    return scores_df,report_df
    #print(report_df.iloc[0])
for classifier in classifiers:
    scores_df, report_df = train_model(classifier,X_train,y_train,X_test,y_test,report_df)  

GaussianNB() accuracy: 0.7650862068965517 precision: 0.7650862068965517 recall:  0.7650862068965517 f1_score:  0.7650862068965517
              precision    recall  f1-score   support

           0       0.99      0.68      0.80       115
           1       0.40      0.98      0.57        42
           2       0.76      0.66      0.70        99
           3       0.77      0.86      0.81        90
           4       0.97      0.80      0.87       118

    accuracy                           0.77       464
   macro avg       0.78      0.79      0.75       464
weighted avg       0.84      0.77      0.78       464

   GaussianNB() class precision recall f1_score support
0  GaussianNB()     0      0.99   0.68     0.80     115
1  GaussianNB()     1      0.40   0.98     0.57      42
2  GaussianNB()     2      0.76   0.66     0.70      99
3  GaussianNB()     3      0.77   0.86     0.81      90
4  GaussianNB()     4      0.97   0.80     0.87     118
KNeighborsClassifier() accuracy: 0.7521551724

In [379]:
print(scores_df)

           GaussianNB()  KNeighborsClassifier()  DecisionTreeClassifier()  \
accuracy       0.765086                0.752155                  0.784483   
precision      0.765086                0.752155                  0.784483   
recall         0.765086                0.752155                  0.784483   
f1_score       0.765086                0.752155                  0.784483   

           RandomForestClassifier()  AdaBoostClassifier()  \
accuracy                   0.864224              0.372845   
precision                  0.864224              0.372845   
recall                     0.864224              0.372845   
f1_score                   0.864224              0.372845   

           LogisticRegression()  
accuracy               0.883621  
precision              0.883621  
recall                 0.883621  
f1_score               0.883621  


In [381]:
scores_df.head()

,GaussianNB(),KNeighborsClassifier(),DecisionTreeClassifier(),RandomForestClassifier(),AdaBoostClassifier(),LogisticRegression()
accuracy,0.765086,0.752155,0.784483,0.864224,0.372845,0.883621
precision,0.765086,0.752155,0.784483,0.864224,0.372845,0.883621
recall,0.765086,0.752155,0.784483,0.864224,0.372845,0.883621
f1_score,0.765086,0.752155,0.784483,0.864224,0.372845,0.883621
